In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

<h1 style="color: red;">Section 1: Data</h1>

<h2>1) Préparation de données</h2>

In [ ]:
np.random.seed(44)  # à chaque exécution, générer le même dataset de manière aléatoire

# Coefficients
a1, a2, b = 2, 3, 5  # y = 2*X1 + 3*X2 + 5 + bruit
nombre_points = 100  # Nombre de points

# Génération des deux features (X1 et X2)
X1 = np.random.rand(nombre_points) * 10
X2 = np.random.rand(nombre_points) * 10

# Empilement des features dans une seule matrice (shape: (100, 2))
X = np.column_stack((X1, X2))

# Génération du bruit
bruit = np.random.randn(nombre_points) * 2  # Bruit

# Calcul de la target
y = a1 * X1 + a2 * X2 + b + bruit

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

### Importance de la division en training set et test set

La division des données en ensembles d'entraînement et de test est cruciale pour :
1. **Évaluer la performance réelle** : Le test set permet d'évaluer comment le modèle généralise sur des données non vues
2. **Détecter le surapprentissage** : Si le modèle performe bien sur les données d'entraînement mais mal sur le test, il y a surapprentissage
3. **Sélection de modèles** : Comparer différents modèles de manière objective
4. **Validation des hyperparamètres** : Optimiser les paramètres sans biaiser l'évaluation finale

<h1 style="color: red;">Section 2: Neural network avec tensorflow</h1>

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

<h2>2) Modèle de réseau de neurones</h2>

### Architecture proposée:
- **Inputs**: 2 features (X1, X2)
- **Couches cachées**: 0 (réseau simple)
- **Couche de sortie**: 1 neurone (régression)
- **Fonction d'activation**: linéaire (pas d'activation pour la régression)
- **Nombre de paramètres**: 2 poids + 1 biais = 3 paramètres

In [ ]:
# Création du modèle
model_nn = Sequential()
output_layer = Dense(1, input_shape=(X_train.shape[1],), activation='linear')
model_nn.add(output_layer)

# Configuration de l'optimiseur
opt = Adam(learning_rate=0.01)

# Compilation du modèle
model_nn.compile(optimizer=opt, loss="mse", metrics=["mse"])

# Affichage du résumé du modèle
model_nn.summary()

# Entraînement du modèle
history = model_nn.fit(X_train, y_train, epochs=4100, verbose=0, validation_split=0.2)

print("\nEntraînement terminé!")

### Que fait la fonction fit()?

La fonction `fit()` :
1. **Entraîne le modèle** en ajustant les poids et biais
2. **Minimise la fonction de coût** (MSE) par descente de gradient
3. **Itère sur les epochs** : répète le processus d'apprentissage
4. **Met à jour les paramètres** à chaque batch
5. **Retourne l'historique** des métriques d'entraînement

In [ ]:
# Calcul manuel du nombre de paramètres
print("=== ANALYSE DES PARAMÈTRES ===")
print(f"Nombre d'inputs: {X_train.shape[1]}")
print(f"Nombre de neurones en sortie: 1")
print(f"Nombre de poids: {X_train.shape[1]} × 1 = {X_train.shape[1]}")
print(f"Nombre de biais: 1")
print(f"Total paramètres (calcul manuel): {X_train.shape[1] + 1}")
print(f"Total paramètres (modèle): {model_nn.count_params()}")

# Extraction des paramètres
W_nn, bias_nn = model_nn.layers[0].get_weights()
print(f"\nPoids (W): {W_nn.flatten()}")
print(f"Biais (b): {bias_nn}")
print(f"\nCoefficients théoriques: a1={a1}, a2={a2}, b={b}")

### Importance du tuning (régularisation)

Le tuning est important dans les cas suivants :
1. **Surapprentissage** : Le modèle mémorise les données d'entraînement
2. **Sous-apprentissage** : Le modèle est trop simple pour capturer les patterns
3. **Données bruitées** : Nécessité de régularisation L1/L2
4. **Datasets complexes** : Optimisation des hyperparamètres
5. **Contraintes computationnelles** : Équilibrer performance et vitesse

<h2>3) Prédiction en utilisant le modèle</h2>

In [ ]:
# Prédictions avec le modèle
yhat_nn = model_nn.predict(X_test)
yhat_nn = yhat_nn.flatten()

print(f"Forme des prédictions: {yhat_nn.shape}")
print(f"Premières prédictions: {yhat_nn[:5]}")

In [ ]:
# Prédiction manuelle sans utiliser predict()
W_manual, bias_manual = model_nn.layers[0].get_weights()
yhat_manual = X_test @ W_manual.flatten() + bias_manual[0]

print("=== COMPARAISON PRÉDICTIONS ===")
print(f"Prédictions model.predict(): {yhat_nn[:3]}")
print(f"Prédictions manuelles: {yhat_manual[:3]}")
print(f"Différence maximale: {np.max(np.abs(yhat_nn - yhat_manual))}")

<h2>4) Evaluation du modèle</h2>

In [ ]:
# Performance sur le test set
mse_nn = mean_squared_error(y_test, yhat_nn)
r2_nn = r2_score(y_test, yhat_nn)

print("=== PERFORMANCE MODÈLE NEURAL ===")
print(f"MSE: {mse_nn:.4f}")
print(f"R²: {r2_nn:.4f}")

# Performance sur le training set
yhat_train = model_nn.predict(X_train).flatten()
mse_train = mean_squared_error(y_train, yhat_train)
r2_train = r2_score(y_train, yhat_train)

print(f"\nMSE Training: {mse_train:.4f}")
print(f"R² Training: {r2_train:.4f}")

# Analyse des résidus
residuals = y_test - yhat_nn

# Graphique des résidus
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.scatter(yhat_nn, residuals, alpha=0.6, color='blue')
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Valeurs prédites (ŷ)')
plt.ylabel('Résidus (y - ŷ)')
plt.title('Graphique des résidus')
plt.grid(True)

plt.subplot(1, 3, 2)
plt.scatter(y_test, yhat_nn, alpha=0.6, color='green')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Valeurs réelles')
plt.ylabel('Valeurs prédites')
plt.title('Prédictions vs Réalité')
plt.grid(True)

plt.subplot(1, 3, 3)
plt.hist(residuals, bins=20, alpha=0.7, color='purple')
plt.xlabel('Résidus')
plt.ylabel('Fréquence')
plt.title('Distribution des résidus')
plt.grid(True)

plt.tight_layout()
plt.show()

### Interprétation des résultats

1. **R² proche de 1** : Le modèle explique bien la variance des données
2. **MSE faible** : Les erreurs de prédiction sont petites
3. **Résidus aléatoires** : Pas de pattern visible = bon modèle
4. **Performance train vs test** : Similaire = pas de surapprentissage

### Pourquoi évaluer sur training ET test set ?
- **Détecter le surapprentissage** : Performance train >> test
- **Valider la généralisation** : Performance similaire = bon modèle
- **Diagnostiquer les problèmes** : Sous-apprentissage si les deux sont mauvais

<h1 style="color: red;">Section 3: Régression linéaire from scratch</h1>

<h2>Modèle de régression linéaire from scratch avec utilisation des matrices</h2>

In [ ]:
# 1) Calculs élémentaires pour une itération
print("=== CALCULS ÉLÉMENTAIRES ===")

# Dataset d'exemple
X_example = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y_example = np.array([[10], [20], [30], [40]])

# Modèle initial
W_example = np.array([[0.2], [0.3]])
bias_example = 0.1
learning_rate_example = 0.01

print(f"X: \n{X_example}")
print(f"y: \n{y_example.flatten()}")
print(f"W initial: \n{W_example.flatten()}")
print(f"bias initial: {bias_example}")

# a) Calcul de la prédiction
y_pred_example = X_example @ W_example + bias_example
print(f"\na) Prédictions: \n{y_pred_example.flatten()}")

# b) Calcul des erreurs
errors_example = y_pred_example - y_example
print(f"\nb) Erreurs: \n{errors_example.flatten()}")

# c) Calcul des gradients
n_example = len(X_example)
dW_example = (1/n_example) * (X_example.T @ errors_example)
db_example = (1/n_example) * np.sum(errors_example)
print(f"\nc) Gradients:")
print(f"dW: \n{dW_example.flatten()}")
print(f"db: {db_example}")

# d) Mise à jour du modèle
W_new = W_example - learning_rate_example * dW_example
bias_new = bias_example - learning_rate_example * db_example
print(f"\nd) Nouveaux paramètres:")
print(f"W nouveau: \n{W_new.flatten()}")
print(f"bias nouveau: {bias_new}")

In [ ]:
# 2) Entraînement complet du modèle from scratch
print("\n=== ENTRAÎNEMENT FROM SCRATCH ===")

learning_rate = 0.0001
epochs = 1000

# Initialisation des paramètres
W = np.array([[0.0], [0.0]])  # Shape: (2, 1)
b = 0.0

# Reshape y_train pour garantir les dimensions adéquates 
y_train_reshaped = y_train.reshape(-1, 1)  # Shape: (n_samples, 1)
n = len(X_train)

# Stockage des coûts pour visualisation
costs = []

# Entraînement (descente de gradient vectorisée)
for epoch in range(epochs):
    # Prédiction
    y_pred = X_train @ W + b  # Shape: (n, 1)
    
    # Calcul de l'erreur
    error = y_pred - y_train_reshaped  # Shape: (n, 1)
    
    # Calcul du coût (MSE)
    cost = (1/(2*n)) * np.sum(error**2)
    costs.append(cost)

    # Calcul des gradients
    dW = (1/n) * (X_train.T @ error)  # Shape: (2, 1)
    db = (1/n) * np.sum(error)        # Scalaire

    # Mise à jour des paramètres
    W -= learning_rate * dW
    b -= learning_rate * db
    
    if epoch % 200 == 0:
        print(f"Epoch {epoch}, Cost: {cost:.6f}")

# Résultats finaux
print(f"\nParamètres ajustés:")
print(f"W = \n{W}")
print(f"b = {b:.4f}")
print(f"\nCoefficients théoriques: a1={a1}, a2={a2}, b={b}")

In [ ]:
# 3) Évaluation du modèle from scratch
print("=== ÉVALUATION MODÈLE FROM SCRATCH ===")

# Prédictions sur le test set
yhat_scratch = X_test @ W + b
yhat_scratch = yhat_scratch.flatten()

# Métriques
mse_scratch = mean_squared_error(y_test, yhat_scratch)
r2_scratch = r2_score(y_test, yhat_scratch)

print(f"MSE (from scratch): {mse_scratch:.4f}")
print(f"R² (from scratch): {r2_scratch:.4f}")

# Comparaison avec sklearn
from sklearn.linear_model import LinearRegression
lr_sklearn = LinearRegression()
lr_sklearn.fit(X_train, y_train)
yhat_sklearn = lr_sklearn.predict(X_test)

mse_sklearn = mean_squared_error(y_test, yhat_sklearn)
r2_sklearn = r2_score(y_test, yhat_sklearn)

print(f"\n=== COMPARAISON ===")
print(f"MSE - Neural Network: {mse_nn:.4f}")
print(f"MSE - From Scratch: {mse_scratch:.4f}")
print(f"MSE - Sklearn: {mse_sklearn:.4f}")
print(f"\nR² - Neural Network: {r2_nn:.4f}")
print(f"R² - From Scratch: {r2_scratch:.4f}")
print(f"R² - Sklearn: {r2_sklearn:.4f}")

# Visualisation de la convergence
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(costs)
plt.title('Convergence du coût')
plt.xlabel('Epochs')
plt.ylabel('MSE Cost')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.scatter(y_test, yhat_scratch, alpha=0.6, label='From Scratch', color='red')
plt.scatter(y_test, yhat_sklearn, alpha=0.6, label='Sklearn', color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Valeurs réelles')
plt.ylabel('Valeurs prédites')
plt.title('Comparaison des modèles')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()